# Image orientation classification using MobileNet model

## To run this notebook you need:

0. All images must be in 75x75 resolution and RGB format (use C:\Users\User\DataScience\area51\notebooks\3_prepare_data.ipynb to prepare data)




In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


set()

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense

# Define input shape (75x75 RGB images)
input_shape = (75, 75, 3)

# Input layer
inputs = Input(shape=input_shape)

# Load MobileNet without top classification layer
base_model = MobileNet(
    input_tensor=inputs,
    weights='imagenet',
    include_top=False  # we'll add our own head
)

# Freeze base model for transfer learning (optional)
base_model.trainable = False

# Custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
outputs = Dense(4, activation='softmax')(x)  # 4 orientation classes

# Final model
model = Model(inputs=inputs, outputs=outputs)

# Compile
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Summary
model.summary()




In [19]:

a={1,1,3, 4,5}; b={2,3}; c={1,5}
b-c, b-a




({2, 3}, {2})

In [ ]:
class _OrientationEstimatorBase():

    def __init__(self, model_name, **kwargs):

        # 1. protected basic variables
        # define allowed parameter keys
        self.__basic_keys = {
            "model_prefix",
            "model_dir",
            "data_dir",
            "train_valid_dataset_dir",
            "test_dataset_dir",
            "test_dataset_desc",
            "train_ds",
            "val_ds",
            "history",
            "model",
            "estimated_data_results",
            "estimated_test_results"
        }
        self._model_prefix = model_name
        self._model_dir = rf"C:\Users\User\DataScience\area51\models\resnet\{model_name}"
        self._data_dir = r"C:\Users\User\DataScience\area51\data\COVID-19_Radiography_Dataset"
        self._train_valid_dataset_dir    =   r"C:\Users\User\DataScience\area51\data_224x224\train_val_224x224"
        self._test_dataset_dir           =   r"C:\Users\User\DataScience\area51\data_224x224\test_224x224"
        self._test_dataset_desc = rf'''
                                        test dataset dir: {self._test_dataset_dir}

                                        dataset contains images:

                                            # rotation    :       image count
                                            # 0°          :       12
                                            # +90         :       13
                                            # -90         :       13
                                            # 180°        :       14
                                        '''
        self._train_ds, self._val_ds = None, None
        self._history = None
        self._model = None
        self._estimated_data_results = None
        self._estimated_test_results  = None

        # 2. parameters for image_dataset_from_directory
        # --> a set object ensures the uniqness of a key
        self.__param_set_keys = {
            "directory",            # Path to the root folder containing subfolders of images
            "labels",               # "inferred", list of labels, or None
            "label_mode",           # "int", "categorical", "binary", or None
            "class_names",          # Optional list of class subdirectory names
            "color_mode",           # "rgb" or "grayscale"
            "batch_size",           # Number of images per batch (default: 32)
            "image_size",           # Tuple of (height, width)
            "shuffle",              # Whether to shuffle the data (default: True)
            "seed",                 # Random seed for shuffling and transformations
            "validation_split",     # Float between 0 and 1 for validation split
            "subset",               # "training" or "validation" if using validation_split
            "interpolation",        # Interpolation method (e.g., "bilinear", "nearest")
            "follow_links",         # Whether to follow symlinks inside class subdirectories
            "crop_to_aspect_ratio"  # If True, center-crop images to preserve aspect ratio
        }

        self.__param_set = { 
            "directory" : self._train_valid_dataset_dir,
            "batch_size" : 64,
            "seed": 1,
            "label_mode": "categorical",
            "color_mode": "rgb",        # resNet requires 3 channel otherwise needs to be trained from scratch 
            "image_size": (224, 224),   # img_height, img_width
            "shuffle": True,
            "validation_split": 0.2,
            "labels": "inferred"        #from folder structure
        }

        self.__compiling_keys = {
            "optimizer", "loss", "metrics"
        }

        # 3. Compilation parameters for functional/sequential models
        self.__compiling_params = {
            "optimizer": 'adam',
            "loss": "categorical_crossentropy", #loss function
            "metrics": ['accuracy']
        }

In [ ]:
from tensorflow.keras.applications.mobilenet import preprocess_input

def preprocess(image, label):
    image = tf.image.resize(image, [75, 75])
    image = preprocess_input(image)
    return image, label

train_ds = train_ds.map(preprocess)
val_ds = val_ds.map(preprocess)

